In [1]:
!!pip install pytrends

['Collecting pytrends',
 '  Downloading pytrends-4.7.3-py3-none-any.whl (14 kB)',
 'Requirement already satisfied: pandas>=0.25 in /usr/local/lib/python3.7/dist-packages (from pytrends) (1.1.5)',
 'Requirement already satisfied: requests in /usr/local/lib/python3.7/dist-packages (from pytrends) (2.23.0)',
 'Requirement already satisfied: lxml in /usr/local/lib/python3.7/dist-packages (from pytrends) (4.2.6)',
 'Requirement already satisfied: python-dateutil>=2.7.3 in /usr/local/lib/python3.7/dist-packages (from pandas>=0.25->pytrends) (2.8.2)',
 'Requirement already satisfied: numpy>=1.15.4 in /usr/local/lib/python3.7/dist-packages (from pandas>=0.25->pytrends) (1.19.5)',
 'Requirement already satisfied: pytz>=2017.2 in /usr/local/lib/python3.7/dist-packages (from pandas>=0.25->pytrends) (2018.9)',
 'Requirement already satisfied: six>=1.5 in /usr/local/lib/python3.7/dist-packages (from python-dateutil>=2.7.3->pandas>=0.25->pytrends) (1.15.0)',
 'Requirement already satisfied: idna<3,>

In [2]:
#@title us
# pip install pytrends
import pandas as pd
import numpy as np
from pytrends.request import TrendReq
pytrends = TrendReq(hl="en-US", tz=360)
#pytrends = TrendReq(hl="en-GB", tz=360)
recession_list=['riot', 'coup','bankruptcy','collapse','decline','deflation','downturn','civil war','hard times','inflation','stagnation','unemployment','bust','depression','bad times','Cyperattack','Cyperwar','Environmental disaster','Global warming','Warfare','Mass destruction','Crisis','Asteroid','Aliens','Pandemic','Volcanoes','Catastrophic risk','Emergency','Nuclear terrorism','Pandemic','Nuclear war','terrorism','world war']

def gtrends_overtime(full_list, key_ref, save_name="", directory="", category=0, time='all', loc=''):
    #iterate every 4 item in list plus a keyword reference as the relative comparison
    i = 0
    database=pd.DataFrame()
    while i < len(recession_list):


        l = recession_list[i:(i+4)]
        l.append(key_ref)
        l=[w.replace('_', ' ') for w in l]
        #print(l)
        pytrends.build_payload(l, cat=category, timeframe=time, geo=loc, gprop='')
        df_time = pytrends.interest_over_time()
        #df_time.reset_index(inplace=True)
        df_time.drop(df_time.columns[len(df_time.columns)-1],axis=1, inplace=True)
        #print(df_time)
        database=pd.concat([database,df_time],join="outer", axis=1)
        #pd.merge(database,df_time, left_index=True, right_index=True)
        #df_time_name = "gtrends_overtime"+str(save_name)+str((i+4)//4)+".csv"
        #df_time.to_csv(directory+df_time_name, index = False)
        i += 4
    return database
        
data=gtrends_overtime(recession_list, "recession", "_worldwide_",directory="",  category=71, time='today 5-y', loc="US")

data['date']=data.index

data.replace([np.inf, -np.inf], np.nan)
data=data.dropna()

labels=list(data.columns.unique().drop(['date']))

import altair as alt

brush_sel = alt.selection(type='interval', encodings=['x'])

nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['x'], empty='none')

lower=alt.Chart(data).transform_fold(labels).mark_line().encode(
    y=alt.Y('value:Q'),
    x='date:T',
    color='key:N'
).properties(width=800,height=100).add_selection(brush_sel)

upper=lower.transform_window(
    rolling_mean='mean(value)',
    frame=[-2, 2]
).mark_bar(opacity=0.9).encode(
    y=alt.Y("value:Q"), #,sort='-y' , stack=None
    x='yearmonthdate(date):O',
    color='key:N',
    
    tooltip=alt.Tooltip(['key:N','value:N']),
).properties(width=800,height=300).transform_filter(brush_sel)

selectors = alt.Chart(data).transform_fold(labels).mark_point().encode(
    x='value:Q',
    color='key:N',
    opacity=alt.value(0),
).add_selection(
    nearest
)

up=alt.layer(upper)


alt.vconcat(up,lower) #.save("risiko.html")

/usr/local/lib/python3.7/dist-packages/altair/utils/data.py:150: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  return {"values": data.to_dict(orient="records")}
